In [1]:
%run Jansen_And_Rit.py
%run BOLD_Model.py
%run OptimiseFunctions.py

import numpy as np
import pickle
from skopt import gp_minimize
np.random.seed(123)

In [2]:
# Original A = 3.25 # excitatory, B = 22.0 # inhibitory, C = 135
# B > A. b < a. 
# b = 50, ad = 50, a = 100
# r_0, r_1, r_2 = 0.56
# mean_firing_threshold = 6
# max_firing_rate = 5

# A, B, C, a, ad, b, r_0, r_1, r_2

eeg_res = gp_minimize(find_eeg_loss, [(2, 7), (15, 30), (80, 150)], n_jobs=-1, n_calls=100)

with open('eeg_res.pkl', 'wb') as f:
    pickle.dump(eeg_res, f)

6 21 128 125 50 61
Creating RawArray with float64 data, n_channels=62, n_times=2000
    Range : 0 ... 1999 =      0.000 ...     1.999 secs
Ready.
Effective window size : 2.000 (s)


c:\Users\stapl\anaconda3\envs\nilearnenv\lib\site-packages\mne\time_frequency\psd.py:54: RuntimeWarning: Mean of empty slice
  spect = np.nanmean(spect, axis=-1)


ValueError: array must not contain infs or NaNs

In [ ]:
bold_res = gp_minimize(find_bold_loss, [(2, 7), (15, 30), (80, 150), (75, 145), (30, 70), (30, 70)], n_jobs=-1, n_calls=100)

with open('bold_res.pkl', 'wb') as f:
    pickle.dump(bold_res, f)

print(get_cached_counts())